In [ ]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pymc3 as pm
import pandas as pd
import os

np.random.seed(123)

%matplotlib inline
plt.style.use('ggplot')

import matplotlib

text_size = 20

matplotlib.rcParams['figure.figsize'] = (15, 10)
matplotlib.rcParams['axes.titlesize'] = text_size
matplotlib.rcParams['axes.labelsize'] = text_size - 2
matplotlib.rcParams['xtick.labelsize'] = text_size - 4
matplotlib.rcParams['ytick.labelsize'] = text_size - 4

# UWAGI:
* `MaskedArray` z `numpy` pozawala wepchnac brakujace wartosci (podobnie `pandas`owy `DataFrame` z wartosciami `NaN`);

In [ ]:
from datetime import datetime
DF_reddb = pd.read_csv('reddb.csv')

# przeprocesuj kolumne `Hour()`
DF_reddb['hour'] = DF_reddb['Hour()'].apply(lambda s: datetime.strptime(s, '%H %d/%m/%Y'))
DF_reddb['day'] = DF_reddb['Hour()'].apply(lambda s: datetime.strptime(s, '%H %d/%m/%Y'))

In [ ]:
# wykresy statow

stats = ['imp', 'index', 'pro_index', 'pro_scroll_8_8', 'con1']

fig = plt.figure(figsize = (30, 5))

for stat_number, stat_name in enumerate(stats):
    ax = fig.add_subplot(1, len(stats), stat_number + 1)
    ax.hist(DF_reddb[stat_name])
    ax.set_xlabel('log10({0})'.format(stat_name))
    if stat_number == 0:
        ax.set_ylabel('log10(count)')
    ax.set_xscale('log')
    ax.set_yscale('log')